上次知识回顾：<a href="http://github.lesschina.com/python/base/ext/基础衍生.html" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9278573.html</a>

终于期末考试结束了，聪明的小明同学现在当然是美滋滋的过暑假了，左手一只瓜，右手一本书～正在给老乡小张同学拓展他研究多日的知识点


## 1.NetCore装饰器模式

装饰器这次从`C#`开始引入，上次刚讲`迭代器模式`，这次把`装饰器模式`也带一波（纯Python方向的可以选择性跳过，也可以当扩展）

其实通俗讲就是，**给原有对象动态的添加一些额外的职责**（毕竟动不动就改类你让其他调用的人咋办？也不符合开放封闭原则是吧～）

举个简单的例子：(<a href="https://github.com/lotapp/BaseCode/tree/master/netcore/3_Ext/Decorators" target="_blank">https://github.com/lotapp/BaseCode/tree/master/netcore/3_Ext/Decorators</a>)

BaseComponent.cs
```csharp
/// <summary>
/// 组件的抽象父类
/// </summary>
public abstract class BaseComponent
{
    /// <summary>
    /// 定义一个登录的抽象方法
    /// 其他方法，这边省略
    /// </summary>
    public abstract string Login();
}
```
LoginComponent.cs
```csharp
/// <summary>
/// 默认登录组件（账号+密码）
/// 其他方法省略
/// 友情提醒一下，抽象类里面可以定义非抽象方法
/// </summary>
public class LoginComponent : BaseComponent
{
    public override string Login()
    {
        return "默认账号密码登录";
    }
}
```
默认调用：
```csharp
static void Main(string[] args)
{
    var obj = new LoginComponent();
    var str = obj.Login();
    Console.WriteLine(str);
}
```

---

如果这时候平台需要添加微信第三方登录，怎么办？一般都是用继承来解决，其实还可以通过灵活的`装饰器`来解决：（好处可以自己体会）


先定义一个通用装饰器（不一定针对登录，注册等等只要在BaseComponent中的都能用）
```csharp
/// <summary>
/// 装饰器
/// </summary>
public class BaseDecorator : BaseComponent
{
    protected BaseComponent _component;
    /// <summary>
    /// 构造函数
    /// </summary>
    /// <param name="obj">登录组件对象</param>
    protected BaseDecorator(BaseComponent obj)
    {
        this._component = obj;
    }
    public override string Login()
    {
        string str = string.Empty;
        if (_component != null) str = _component.Login();
        return str;
    }
}
```
现在根据需求添加微信登录：（符合开放封闭原则）
```csharp
/// <summary>
/// 默认登录组件（账号+密码）
/// 其他方法省略
/// </summary>
public class WeChatLoginDecorator : BaseDecorator
{
    public WeChatLoginDecorator(BaseComponent obj) : base(obj)
    {
    }
    /// <summary>
    /// 添加微信第三方登录
    /// </summary>
    /// <returns></returns>
    public string WeChatLogin()
    {
        return "add WeChatLogin";
    }
}
```
调用：（原有系统该怎么用就怎么用，新系统可以使用装饰器来添加新功能）
```csharp
static void Main(string[] args)
{
    #region 登录模块V2
    // 实例化登录装饰器
    var loginDecorator = new WeChatLoginDecorator(new LoginComponent());
    // 原有的登录方法
    var str1 = loginDecorator.Login();
    // 现在新增的登录方法
    var str2 = loginDecorator.WeChatLogin();
    Console.WriteLine($"{str1}\n{str2}");
    #endregion
}
```
结果：
```
默认账号密码登录
add WeChatLogin
```
---

如果再加入QQ和新浪登录的功能就再添加一个V3版本的装饰器，继承当时V2版本的登录即可（版本迭代特别方便）
```csharp
/// <summary>
/// 默认登录组件（账号+密码）
/// 其他方法省略
/// </summary>
public class LoginDecoratorV3 : WeChatLoginDecorator
{
    public LoginDecoratorV3(BaseComponent obj) : base(obj)
    {
    }

    /// <summary>
    /// 添加QQ登录
    /// </summary>
    /// <returns></returns>
    public string QQLogin()
    {
        return "add QQLogin";
    }

    /// <summary>
    /// 添加新浪登录
    /// </summary>
    /// <returns></returns>
    public string SinaLogin()
    {
        return "add SinaLogin";
    }
}
```
调用：
```csharp
static void Main(string[] args)
{
    #region 登录模块V3
    // 实例化登录装饰器
    var loginDecoratorV3 = new LoginDecoratorV3(new LoginComponent());
    // 原有的登录方法
    var v1 = loginDecoratorV3.Login();
    // 第二个版本迭代中的微信登录
    var v2 = loginDecoratorV3.WeChatLogin();
    // 新增的QQ和新浪登录
    var qqLogin = loginDecoratorV3.QQLogin();
    var sinaLogin = loginDecoratorV3.SinaLogin();
    Console.WriteLine($"{v1}\n{v2}\n{qqLogin}\n{sinaLogin}");
    #endregion
}
```
结果：
```
默认账号密码登录
add WeChatLogin
add QQLogin
add SinaLogin
```

---
其实还有很多用处，比如原有系统缓存这块当时考虑不到，现在并发来了，已经上线了，原有代码又不太敢大幅度修改，这时候装饰器就很方便的给某些功能添加点缓存、测试、日记等等系列功能

实际场景说的已经很明白了，其他的自己摸索一下吧

## 2.Python装饰器

那Python怎么实现装饰器呢？小胖问道。

小明屁颠屁颠的跑过去说道，通过闭包咯～（闭包如果忘了，可以<a href="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#8.闭包" target="_blank">回顾</a>一下）

### 2.1.装饰器引入

来看一个应用场景，以前老版本系统因为并发比较小，没考虑到缓存
```py
def getData():
    print("直接数据库读取数据")

def main():
    getData()

if __name__ == '__main__':
    main()
```
在不修改原有代码的前提下咋办？我们参照C#和Java写下如下代码：

In [1]:
# 添加一个闭包
def cache(func):
    def decorator():
        print("给功能添加了缓存")
        if True:
            pass
        else:
            func()# 如果缓存失效则读取数据库获取新的数据
    return decorator

def getData():
    print("直接数据库读取数据")

def main():
    f1 = cache(getData)
    print(type(f1))
    f1()

if __name__ == '__main__':
    main()

<class 'function'>
给功能添加了缓存


小张问道：“怎么也这么麻烦啊，C#的那个我就有点晕了，怎么Python也这样啊？”`f1 = cache(getData)` `f1()`

小明哈哈一笑道：“人生苦短，我用Python～这句话可不是随便说着玩的，来来来，看看Python的语法糖”：

In [2]:
def cache(func):
    def decorator():
        print("给功能添加了缓存")
        if True:
            pass
        else:
            func()  # 如果缓存失效则读取数据库获取新的数据
    return decorator

@cache
def getData():
    print("直接数据库读取数据")

def main():
    getData()

if __name__ == '__main__':
    main()

给功能添加了缓存


其实
```py
@cache
def getData()
```
等价于
```py
# 把f1改成函数名字罢了。可以这么理解：getData重写指向了一个新函数
getData = cache(getData)
```

小张同学瞪了瞪眼睛，努力回想着以前的知识点，然后脱口而出：“这不是我们之前讲的<a href="https://www.cnblogs.com/dotnetcrazy/p/9202988.html#1.5-装饰器，让方法像属性那样便利" target="_blank">属性装饰器</a>吗？而且好方便啊，这完全符合开放封闭原则啊！“

```py
class Student(object):
    def __init__(self, name, age):
        # 一般需要用到的属性都直接放在__init__里面了
        self.name = name
        self.age = age

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, name):
        self.__name = name

    @property
    def age(self):
        return self.__age

    @age.setter
    def age(self, age):
        if age > 0:
            self.__age = age
        else:
            print("age must > 0")

    def show(self):
        print("name:%s,age:%s" % (self.name, self.age))
```

小明也愣了愣，说道：”也对哦，你不说我都忘了，我们学习面向对象三大特性的时候经常用呢，怪不得这么熟悉呢“

随后又嘀咕了一句：”我怎么不知道开放封闭原则...“

小张嘲笑道：”这你都不知道？**对扩展开放，对已经实现的代码封闭**嘛～“

In [3]:
# 需要注意一点
def cache(func):
    print("装饰器开始装饰")
    def decorator():
            print("给功能添加了缓存")
            if True:
                pass
            else:
                func()  # 如果缓存失效则读取数据库获取新的数据
    return decorator

@cache # 当你写这个的时候，装饰器就开始装饰了，闭包里面的功能是你调用的时候执行
def getData():
    print("直接数据库读取数据")

装饰器开始装饰


### 2.2.多个装饰器

小明赶紧扯开话题，”咳咳，我们接下来我们接着讲装饰器"

小张问道，像上面那个第三方登录的案例，想加多少加多少，Python怎么办呢？

小明一笑而过～

现在项目又升级了，要求每次调用都要打印一下日记信息，方便以后纠错，小张先用自己的理解打下了这段代码，然后像小明请教：

In [4]:
def log(func):
    def decorator():
        print("输出日记信息")
        cache(func)()
    return decorator
    
def cache(func):
    def decorator():
        print("给功能添加了缓存")
        if True:
            pass
        else:
            func()  # 如果缓存失效则读取数据库获取新的数据
    return decorator

@log
def getData():
    print("直接数据库读取数据")

def main():
    getData()

if __name__ == '__main__':
    main()

输出日记信息
给功能添加了缓存


小明刚美滋滋的喝着口口可乐呢，看到代码后一不小心喷了小张一脸，然后尴尬的说道：“Python又不是只能装饰一个装饰器，来看看我的代码”：

In [5]:
def log(func):
    print("开始装饰Log模块")
    def decorator():
        print("输出日记信息")
        func()
    return decorator

def cache(func):
    print("开始装饰Cache模块")
    def decorator():
        print("给功能添加了缓存")
        if True:
            pass
        else:
            func()  # 如果缓存失效则读取数据库获取新的数据
    return decorator

@log
@cache
def getData():
    print("直接数据库读取数据")

def main():
    getData()

if __name__ == '__main__':
    main()

开始装饰Cache模块
开始装饰Log模块
输出日记信息
给功能添加了缓存


小张耐心的看完了代码，然后说道：“咦，我发现它**装饰的时候是从下往上装饰，执行的时候是从上往下**啊？执行的时候程序本来就是从上往下，按照道理应该是从上往下装饰啊？”

小明神秘的说道：“你猜啊～你可以把它理解为**寄快递和拆快递**”

小张兴奋的跳起来了：

**装饰器：装快递，先包装里面的物品，然后再加个盒子。执行装饰器：拆快递，先拆外面的包装再拆里面的**～简直妙不可言啊

### 2.3.带参装饰器

小明继续讲述他哥哥的血泪历史：

需求时刻在变，系统使用范围更广了，为了不砸场子，抠门的老板决定每年多花5W在技术研发的硬件支持上，这下子技术部老开心了，想想以前前端只能通过CDN和HTTP请求来缓存，后端只能依赖页面缓存和数据库缓存就心塞，于是赶紧新增加一台Redis的云服务器。为了以后和现在缓存代码得变一变了，**需要支持指定的缓存数据库**：（如果不是维护别人搞的老项目，你这么玩保证被打死，开发的时候老老实实的工厂模式搞起）

In [6]:
# 可以理解为，在原来的外面套了一层
def cache(cache_name):
    def decorator(func):
        if cache_name == "redis":
            def inner():
                print("给功能添加了Redis缓存")
                if True:
                    pass
                else:
                    func()  # 如果缓存失效则读取数据库获取新的数据
            return inner
        elif cache_name == "memcache":
            pass
        else:
            pass  # 默认页面缓存
    return decorator

@cache("redis") # 相当于是：getData = cache(”redis“)(getData)
def getData():
    print("直接数据库读取数据")

def main():
    getData()

if __name__ == '__main__':
    main()

给功能添加了Redis缓存


小张很高兴，然后练了练手，然后质问小明道：”你是不是藏了一手！“

代码如下：

In [7]:
def log(func):
    def inner():
        print("%s log_info..." % func.__name__)
        func()
    return inner

@log
def login_in(name_str, pass_str):
    return "欢迎登录：%s" % (name_str)

@log
def login_out():
    print("已经退出登录")

@log
def getData(id):
    print("%s:data xxx" % id)

def main():
    login_out()
    getData(1)
    print(login_in("小明", "xxx"))

if __name__ == '__main__':
    main()

login_out log_info...
已经退出登录


TypeError: inner() takes 0 positional arguments but 1 was given

### 2.4.通用装饰器

小明尴尬的笑了下，然后赶紧倾囊相授，定义一个**通用的装饰器**：（传参数就在外面套一层）
```py
def log(func):
    # 可变参 + 关键字参数
    def inner(*args,**kv):
        print("%s log_info..." % func.__name__)
        return func(*args,**kv)
    return inner
```
这部分知识如果忘记了可以回顾一下，我们之前讲的函数系列：<a href="https://www.cnblogs.com/dotnetcrazy/p/9175950.html#2.3.关键字参数" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9175950.html</a>

In [8]:
def log(func):
    # 可变参 + 关键字参数
    def inner(*args,**kv):
        print("%s log_info..." % func.__name__)
        return func(*args,**kv)
    return inner

@log
def login_in(name_str, pass_str):
    return "欢迎登录：%s" % (name_str)

@log
def login_out():
    print("已经退出登录")

@log
def getData(id):
    print("%s:data xxx" % id)

def main():
    login_out()
    getData(1)
    print(login_in("小明", "xxx"))

if __name__ == '__main__':
    main()

login_out log_info...
已经退出登录
getData log_info...
1:data xxx
login_in log_info...
欢迎登录：小明


## 3.